# BUILD THE NEURAL NETWORK
  
Neural networks comprise of layers/modules that perform operations on data.   
comprise of : ~로 구성되는  
신경망은 데이터에 대한 연산을 수행하는 층/모듈로 구성됨  
  
The torch.nn namespace provides all the building blocks you need to build your own neural network.   

Every module in PyTorch subclasses the nn.Module.  
PyTorch의 각 모듈은 nn.Module을 서브클래스화  
하위 클래스가 상위 클래스를 상속받아 새로운 클래스를 정의하는 것  
상위 클래스는 일반적인 특성이나 기능을 정의하고, 하위 클래스는 이러한 기능을 상속받아 추가하거나 수정하여 특정한 동작을 수행하도록 만든다  
PyTorch의 nn.Module은 신경망 모델의 기본 클래스로 사용됨  
사용자는 이 클래스를 서브클래스화하여 자신만의 신경망 아키텍처를 정의할 수 있음  
(기존의 기능을 상속받아 사용하면서 필요한 추가적인 동작을 구현)  

A neural network is a module itself that consists of other modules (layers).   
This nested structure allows for building and managing complex architectures easily.  
    
In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.  

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training
  
We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available.   
Let’s check to see if torch.cuda or torch.backends.mps are available, otherwise we use the CPU.  

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


### Define the Class
  
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__.   
  
Every nn.Module subclass implements the operations on input data in the forward method.  

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. 모델 사용을 위해 입력 데이터를 전달  
This executes the model’s forward, along with some background operations.   
execute : 실행하다  
Do not call model.forward() directly!  
  
Calling the model on the input returns   
a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class,   
and dim=1 corresponding to the individual values of each output.  
corresponding : ~에 해당하는   
dim=0은 각 클래스에 대한 10개의 원시 예측 값 출력에 해당  
dim=1은 각 출력의 개별 값에 해당하는 2차원 텐서가 반환  
  
We get the prediction probabilities by passing it through an instance of the nn.Softmax module.  

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


---

### Model Layers

Let’s break down the layers in the FashionMNIST model.   
break down : (분석할 수 있도록) 나누어지다  
각 레이어에서 어떤 일이 일어나는지 나누어서 분석하겠다는 뜻  
이외에 고장나다, 실패하다, 나빠지다, 감정을 주체하지 못하다 라는 뜻도 있음  

To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten
  
We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).  
contiguous : 인접한, 근접한  
각 2D 28x28 이미지를 784개의 픽셀 값으로 이루어진 연속 배열로 변환하기 위해 nn.Flatten 레이어를 초기화  
(dim=0에서 미니배치 차원은 유지)  

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.  
저장된 가중치(weights)와 편향(biases)을 사용하여 입력에 선형 변환을 적용하는 모듈  

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU
  
Non-linear activations are what create the complex mappings between the model’s inputs and outputs.   
They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.  
  
In this model, we use nn.ReLU between our linear layers,   
but there’s other activations to introduce non-linearity in your model.  

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2379,  0.5066,  0.0010, -0.3927,  0.3844, -0.0021, -0.4730,  0.1977,
          0.2838,  0.5093,  0.0119, -0.0736, -0.3844,  0.1870,  0.2491, -0.1793,
          0.4579, -0.2174,  0.0553, -0.1347],
        [-0.1555,  0.5054, -0.0987, -0.2066, -0.1550, -0.4495, -0.6629,  0.0096,
          0.2689,  0.6671, -0.0037, -0.0773, -0.2050,  0.0640,  0.2527,  0.2271,
          0.6222,  0.2373, -0.0578,  0.1462],
        [ 0.1884,  0.5414, -0.3695, -0.2746,  0.0301,  0.0496, -0.5556, -0.1155,
          0.0947,  0.7566, -0.0095, -0.2681, -0.4242,  0.1492,  0.6983, -0.0166,
          0.3179,  0.0968, -0.1286,  0.0912]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2379, 0.5066, 0.0010, 0.0000, 0.3844, 0.0000, 0.0000, 0.1977, 0.2838,
         0.5093, 0.0119, 0.0000, 0.0000, 0.1870, 0.2491, 0.0000, 0.4579, 0.0000,
         0.0553, 0.0000],
        [0.0000, 0.5054, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0096, 0.2689,
         0.6671, 0.0000, 0.0000, 0.0000, 0.0640, 0.25

#### nn.Sequential
  
nn.Sequential is an ordered container of modules. 
순서가 있는 자료구조    
The data is passed through all the modules in the same order as defined.   
You can use sequential containers to put together a quick network like seq_modules.  

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax
  
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module.   
The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class.   
dim parameter indicates the dimension along which the values must sum to 1.

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters
  
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training.   
Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.  
  
In this example, we iterate over each parameter, and print its size and a preview of its values.

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0213, -0.0237, -0.0108,  ..., -0.0021, -0.0182, -0.0299],
        [-0.0206,  0.0113,  0.0107,  ...,  0.0220, -0.0348,  0.0211]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0320, -0.0091], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0049, -0.0298,  0.0420,  ..., -0.0248,  0.0437,  0.0174],
        [-0.0317,  0.0384,  0.0015,  ...,  0.0282, -0.0188,  0.0087]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 